Price Predictor using ML
##Atrributes of the data

1. CRIM: per capita crime rate by town
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS: proportion of non-retail business acres per town
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
5. NOX: nitric oxides concentration (parts per 10 million)
6. RM: average number of rooms per dwelling
7. AGE: proportion of owner-occupied units built prior to 1940
8. DIS: weighted distances to five Boston employment centres
9. RAD: index of accessibility to radial highways
10. TAX: full-value property-tax rate per $10,000
11. PTRATIO: pupil-teacher ratio by town
12. B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13. LSTAT: % lower status of the population
14. MEDV: Median value of owner-occupied homes in

In [ ]:
#import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split #used for spliting dataset into training and testing
from sklearn import metrics
from sklearn.model_selection import cross_val_score
#import sklearn.preprocessing, sklearn.decomposition, \sklearn.linear_model, sklearn.pipeline, sklearn.metrics, \sklearn.compose
# import sklearn_pandas


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
feature_names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","MEDV"]
#reading data in csv file and creating a dataframe
csv_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Boston_house_price_data.csv",names=feature_names)
csv_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


##Analyzing the Data

In [ ]:
# Steps
# 1) csv_df.info()
# 2) csv_df['column_name'].value_counts()
# 3) csv_df.describe()
# 4) csv_df.hist(bins=50,figsize=(20,10))
## Here we also seperate our data into features and labels to simplify the process of splitting
labels_data=csv_df['MEDV'].copy(deep=True)
features_data=csv_df.drop('MEDV',axis=1).copy(deep=True)

In [ ]:
#here we done stratified sampling on the basis of 'CHAS' feature
features,features_test,labels,labels_test=train_test_split(features_data,labels_data,test_size=0.2,train_size=0.8,random_state=42,shuffle=True,stratify=features_data['CHAS'])
X_test=features_test.to_numpy()
Y_test=labels_test.to_numpy()
#From here on we do not touch our test datas we only work on our trianing data

In [ ]:
# We also created a fulldata_train by adding features and labels for future operations
fulldata_train=features.join(labels).copy(deep=True)
#finding and plottinf correlation matrix
corr_matrix=fulldata_train.corr()
#heatmap plotting using seaborn
'''
import seaborn as sns
sns.heatmap(corr_matrix.loc[:,'MEDV':])
'''
#scatter_matrix plotting using pandas.plotting
'''
from pandas.plotting import scatter_matrix
#its better that you use attributes and see plots and then change items in it and again see plots its better otherwise we can see heatmap
attributes=['CRIM','RM','AGE','MEDV']
scatter_matrix(fulldata_train[attributes],alpha=0.8,figsize=(12,10))
plt.show()
'''

"\nfrom pandas.plotting import scatter_matrix\n#its better that you use attributes and see plots and then change items in it and again see plots its better otherwise we can see heatmap\nattributes=['CRIM','RM','AGE','MEDV']\nscatter_matrix(fulldata_train[attributes],alpha=0.8,figsize=(12,10))\nplt.show()\n"

In [ ]:
'''
# we can do this in pipeline
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy="median")
imputer.fit(fulldata_train)
imputed_array=imputer.transform(fulldata_train)
tranformed_df=pd.DataFrame(imputed_array,columns=fulldata_train.columns)
'''

'\n# we can do this in pipeline\nfrom sklearn.impute import SimpleImputer\nimputer=SimpleImputer(strategy="median")\nimputer.fit(fulldata_train)\nimputed_array=imputer.transform(fulldata_train)\ntranformed_df=pd.DataFrame(imputed_array,columns=fulldata_train.columns)\n'

In [ ]:
'''
# we can do this in pipeline

from sklearn.preprocessing import StandardScaler
standard_scaler=StandardScaler()
features=pd.DataFrame(standard_scaler.fit_transform(features),column=features.columns)
features.head()
'''

'\n# we can do this in pipeline\n\nfrom sklearn.preprocessing import StandardScaler\nstandard_scaler=StandardScaler()\nfeatures=pd.DataFrame(standard_scaler.fit_transform(features),column=features.columns)\nfeatures.head()\n'

In [ ]:
# We will crete a pipeline here and add Imputer and scaling into pipeline
my_pipeline=Pipeline([
('imputer',SimpleImputer(strategy="median")),#We can add as many process as we want
('std_scaler',StandardScaler())
])
'''
# Parllel plotting of dataframe
fulldata_train=pd.DataFrame(my_pipeline.fit_transform(fulldata_train),columns=fulldata_train.columns)
ax=pd.plotting.parallel_coordinates(fulldata_train,'MEDV')
legend =ax.legend()
legend.remove()
plt.show()
'''
# fit and transform the pipeline on features
#we are just giving a new name to the features as X_train and labels to Y_train
X_train=my_pipeline.fit_transform(features)
Y_train=labels.to_numpy()

# this below step is very important because if our training feature have passed through pipeline then our testing features should also
# pass through pipeline
X_test=my_pipeline.fit_transform(X_test)

In [ ]:
# import your model and start modelling
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.linear_model import SGDRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor()
model.fit(X_train,Y_train)
# model.score(features,labels)
# model.coef_
# model.intercept_
Y_predict=model.predict(X_test)

# calulating root mean squared error
rmse=np.sqrt(metrics.mean_squared_error(Y_test,Y_predict))
print("Rmse = ",rmse)

#Calculating K-fold cross validation (to check if our model got overfitted)
rmse_cross_val_scores=np.sqrt(-1*cross_val_score(model,X_train,Y_train,scoring="neg_mean_squared_error",cv=10))
print("Rmse cross val scores = \n",rmse_cross_val_scores)

Rmse =  3.4157876348898903
Rmse cross val scores = 
 [2.87568795 2.69911473 4.38115512 2.62964802 3.49147949 2.37951013
 5.09990477 3.2238923  3.17132684 3.35519809]
